In [5]:
import os
import numpy as np
import pandas as pd 

PATH = "data/sampleDataset/delay2024-06-23To2024-07-27.csv"

original_delay_df = pd.read_csv(PATH, sep=",", header=0)
original_delay_df.head(10)

,Date,Route,Time,Day,Location,Incident,Min Delay,Min Gap,Direction,Vehicle
0,2024-06-23,353,02:30,Sunday,NEILSON AND CROW TRAIL,Operations - Operator,30,60,E,3124
1,2024-06-23,334,02:38,Sunday,KENNEDY STATION,Security,30,60,NaN,8749
2,2024-06-23,353,02:42,Sunday,OASIS AND SEASONS,Operations - Operator,30,60,W,3124
3,2024-06-23,363,02:45,Sunday,CNE LOOP EASTERN ENTRA,Mechanical,30,60,NaN,7068
4,2024-06-23,320,02:46,Sunday,YONGE AND YORK MILLS,General Delay,0,0,NaN,0
5,2024-06-23,320,03:04,Sunday,STEELES LOOP,Cleaning - Unsanitary,5,10,S,1218
6,2024-06-23,320,03:23,Sunday,YONGE AND BELMONT,Security,5,10,N,8341
7,2024-06-23,353,03:38,Sunday,STEELES AND MURRAY ROS,Operations - Operator,30,60,W,3124
8,2024-06-23,353,04:15,Sunday,NW GATE AND STEELES,Security,30,60,S,3132
9,2024-06-23,64,05:44,Sunday,BIRCHMOUNT AND DANFORT,General Delay,25,0,E,8525


In [6]:
delay_df = original_delay_df.copy()
def day_to_service_id(day):
  if day in ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']:
    return '1'
  elif day == 'Saturday':
    return '2'
  elif day == 'Sunday':
    return '3'
  else:
    return np.nan
  
# Encode Day to service_id
print('Rows that do not follow the Day shema')
display(delay_df[~delay_df['Day'].isin(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])])
delay_df['Day'] = delay_df['Day'].apply(day_to_service_id)

# Change Route from string to int type
delay_df['Route'] = pd.to_numeric(delay_df['Route'], errors='coerce').fillna(0).astype(np.int64)

# Drop any rows that were not encoded properly
print('Rows that do not follow the Route schema')
display(original_delay_df[delay_df['Route'] == 0])
delay_df = delay_df[delay_df['Day'].notna() & delay_df['Route'] != 0]

# Change Route to string and change Day to service_id
delay_df['Route'] = delay_df['Route'].astype(str)
delay_df = delay_df.rename(columns={'Day':'service_id'})

Rows that do not follow the Day shema


,Date,Route,Time,Day,Location,Incident,Min Delay,Min Gap,Direction,Vehicle


Rows that do not follow the Route schema


,Date,Route,Time,Day,Location,Incident,Min Delay,Min Gap,Direction,Vehicle
10,2024-06-23,NaN,05:44,Sunday,STREETCAR WAY - 831,Security,0,0,NaN,0
43,2024-06-23,NaN,11:14,Sunday,QUEENSWAY GARAGE,Collision - TTC,0,0,E,8165
181,2024-06-23,LINE 1,01:06,Sunday,WOOD AND YONGE,Collision - TTC,0,0,E,1413
249,2024-06-24,NaN,12:54,Monday,WILSON TRAINING CENTRE,Operations - Operator,0,0,NaN,0
471,2024-06-25,NaN,15:07,Tuesday,580 COMMISSIONER STREE,Operations - Operator,0,0,NaN,0
...,...,...,...,...,...,...,...,...,...,...
5643,2024-07-24,LINE 1,22:30,Wednesday,SHEPPARD WEST STATION,Operations - Operator,0,0,NaN,1202
5645,2024-07-24,RAD 600,22:48,Wednesday,MCCOWAN AND STEELES,Mechanical,3,3,N,3101
5685,2024-07-25,NaN,08:21,Thursday,HOLIDAY INN - 600 DIXO,Utilized Off Route,0,0,NaN,7018
5781,2024-07-25,NaN,19:12,Thursday,WARDEN AND BRIDLETOWNE,Diversion,16,34,N,0


In [7]:
delay_dfs = {
  route: 
    {service_id: group for service_id, group in sub_delay_df.groupby('service_id')}
    for route, sub_delay_df in delay_df.groupby('Route')
  }

delay_df.to_pickle('processedD' + os.path.basename(PATH)[1:][:-4] + '.pkl')